In [ ]:
# pip install DeepFace

In [1]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import time

from deepface import DeepFace

- it technically does the job, however its quite slow.

In [ ]:
# Define emotion labels
emotion_labels = ['angry', 'disgust', 'fear', 'happy', 'sad', 'surprise', 'neutral']
text_color = (255, 255, 255)

# Start capturing video
cap = cv2.VideoCapture(0)

prev_frame_time = 0
# used to record the time at which we processed current frame
new_frame_time = 0

while True:
    # Capture frame-by-frame
    ret, frame = cap.read()

    
#     result = emo_detector.detect_emotions(frame)
    try:
        result = DeepFace.analyze(frame, actions=["emotion", "age", "gender", "race"], silent=True)
    
    except ValueError:
        continue
    
    bbox = result[0]["region"]
    emotions = result[0]["emotion"]
    dominant_emotion = result[0]["dominant_emotion"]
    age = result[0]["age"]
    gender = result[0]["dominant_gender"]
    races = result[0]["race"]
    dominant_race = result[0]["dominant_race"]
    
    new_frame_time = time.time()
    
    # fps will be number of frame processed in given time frame
    # since their will be most of time error of 0.001 second
    # we will be subtracting it to get more accurate result
    fps = 1/(new_frame_time-prev_frame_time)
    prev_frame_time = new_frame_time
    
    # Draw rectangle around face and label with predicted emotion
#     cv2.rectangle(frame, (0, 0), (200, 200), (0, 0, 0), -1)
    
    # face bounding box
    x, y, w, h = bbox["x"], bbox["y"], bbox["w"], bbox["h"]
    cv2.rectangle(frame,(x,y),(x+w,y+h),(255,0,0),2)
    
    # emotion scores
    for idx, em in enumerate(emotion_labels):
        text = f"{em}: {emotions[em]:.4f}"
        cv2.putText(frame, text, (10, 20*(idx+1)), cv2.FONT_HERSHEY_SIMPLEX, 0.5,text_color, 2)
        
    cv2.putText(frame, f"Emotion: {dominant_emotion} - {emotions[dominant_emotion]:.4f}",
            (10, 20*(idx+2)), cv2.FONT_HERSHEY_SIMPLEX, 0.5,text_color, 2)

    cv2.putText(frame, f"age: {age}", (10, 20*(idx+3)), cv2.FONT_HERSHEY_SIMPLEX, 0.5,text_color, 2)    
    cv2.putText(frame, f"gender: {gender}", (10, 20*(idx+4)), cv2.FONT_HERSHEY_SIMPLEX, 0.5,text_color, 2)    
    cv2.putText(frame, f"race: {dominant_race}", (10, 20*(idx+5)), cv2.FONT_HERSHEY_SIMPLEX, 0.5,text_color, 2)
    
    cv2.putText(frame, "Press \'q\' to exit", (500, 465), cv2.FONT_HERSHEY_SIMPLEX, 0.5,(255, 255, 255), 2)
    
    cv2.putText(frame, f"fps: {fps:.3f}", (5, 465), cv2.FONT_HERSHEY_SIMPLEX, 0.5,text_color, 2)
    
    # Display the resulting frame
    cv2.imshow('Real-time Emotion Detection', frame)
    
    
    # Press 'q' to exit
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the capture and close all windows
cap.release()
cv2.destroyAllWindows()